# Desk sensor data to occupancy

This notebook is used to convert sensor data to desk occupancy data. 

the datadump contains information on all kinds of sensors, in this notebook we only consider the desk sensors and their occupancy state. 

The sensor data contains the timestamps on which the state of occupancy changes. For analysis purposes, I want to have the occupancy for a timeframe of 15 minutes instead of the instantaneous occupancy at a certain timestamp. 

## Assumptions

the desk sensor data is csv files with one file per day. We assume all occupancy states to reset to unoccupied at midnight. The .csv files have the following format:

| time | deviceName | fCnt | application | msgSource | gatewayMac | occupied | temC | humidity | co2 | sound | lux |

We filter to include entries where application contains the string "desk". Matching on cases could be more performant, but I'm unsure what application names are used. 

*time* is the message timestamp in posixct format. 
*deviceName* is the name of the desk sensor.
*fCnt* is a rolling counter for the messages sent by the sensor, which resets on reconnect. Missing values indicate dropped/lost messages
*application* is the application name of the sensor. We filter to include only desk sensors.
*occupied* is the occupancy state of the desk sensor, with 1 indicating occupied and 0 indicating unoccupied.

Other columns are not relevant for this analysis.

All sensors send out a 'heartbeat' message every ~12 hours. This message contains the last know occupancy state, which should result in consecutive messages with the same occupancy state. We will filter these out.

Unoccupancy is defined as no activity for 30 minutes. Therefore we will subtract 30 minutes from the timestamp of unoccupancy. 


# Aims

Calculating the overall occupancy over all desk sensors per 15-minute intervals. If occupation does not span the entire interval, the ratio occupied in the interval is used. 


The sensor_export_chris directory contains two folders which should not be considered: "archive" and "captainExport_NL_UTC_UTRECHT". All other folders in the sensor_export_directory contain the sensor logs per year. The individual files per day are named "export_assetData-NL-UTC-UTRECHTLROF_YYYYMMDD.csv"

In [16]:
import pandas as pd
import numpy as np

# Load the CSV file
file_path = 'sensor_export_chris/NL-UTC-UTRECHTLROF_2025/export_assetData_NL-UTC-UTRECHTLROF_20250121.csv'
data = pd.read_csv(file_path, sep=';', encoding='utf-8', low_memory=False,skiprows=1)

print(data.columns)

Index(['time', 'deviceName', 'fCnt', 'application', 'msgSource',
       'gatewayMacBest', 'occupied', 'tempC', 'humidity', 'co2', 'sound',
       'lux'],
      dtype='object')


In [20]:
import pandas as pd
import numpy as np

# Load the CSV file
file_path = 'sensor_export_chris/NL-UTC-UTRECHTLROF_2025/export_assetData_NL-UTC-UTRECHTLROF_20250125.csv'
data = pd.read_csv(file_path, sep=';', encoding='utf-8', low_memory=False,skiprows=1)

# Filter to include only desk sensors
data = data[data['application'].str.contains('desk', case=False, na=False)]

# Convert timestamp column to datetime format
data['time'] = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S,%f')

# Sort data by timestamp
data = data.sort_values(by='time')

# Remove consecutive messages with the same occupancy state (heartbeat messages)
data = data[data['occupied'].diff() != 0]

# Subtract 30 minutes from the timestamp of unoccupancy
data.loc[data['occupied'] == 0, 'time'] -= pd.Timedelta(minutes=30)

# Create a new DataFrame to store occupancy per 15-minute intervals
start_time = data['time'].min().replace(hour=0, minute=0, second=0, microsecond=0)
end_time = data['time'].max().replace(hour=23, minute=59, second=59, microsecond=999999)
time_range = pd.date_range(start=start_time, end=end_time, freq='15T')
occupancy_data = pd.DataFrame({'time': time_range, 'occupied': 0})

# Calculate occupancy for each interval
for i in range(len(occupancy_data) - 1):
    interval_start = occupancy_data.loc[i, 'time']
    interval_end = occupancy_data.loc[i + 1, 'time']
    interval_data = data[(data['time'] >= interval_start) & (data['time'] < interval_end)]
    
    if not interval_data.empty:
        occupied_time = (interval_data['time'].diff().fillna(pd.Timedelta(seconds=0)) * interval_data['occupied']).sum()
        occupancy_ratio = occupied_time / (interval_end - interval_start)
        occupancy_data.loc[i, 'occupied'] = occupancy_ratio

# Save the aggregated occupancy data to a new CSV file
output_file_path = 'aggregated_occupancy_20250121.csv'
occupancy_data.to_csv(output_file_path, index=False, sep=';', encoding='utf-8')

print(f"Aggregated occupancy data has been saved to {output_file_path}.")


Aggregated occupancy data has been saved to aggregated_occupancy_20250121.csv.


C:\Users\STBROUWE\AppData\Local\Temp\ipykernel_13620\3343506964.py:26: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  time_range = pd.date_range(start=start_time, end=end_time, freq='15T')
C:\Users\STBROUWE\AppData\Local\Temp\ipykernel_13620\3343506964.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9809088888888889' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  occupancy_data.loc[i, 'occupied'] = occupancy_ratio


In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Define the directory containing the sensor data
sensor_export_directory = 'sensor_export_chris/NL-UTC-UTRECHTLROF_2025'






# Function to process a single CSV file
def process_csv(file_path):
    # Extract the date from the filename
    file_date_str = os.path.basename(file_path).split('_')[-1].split('.')[0]
    file_date = datetime.strptime(file_date_str, '%Y%m%d').date()
    
    # Define data types for the relevant columns
    dtype = {
        'time': str,
        'deviceName': str,
        'fCnt': 'Int64',
        'application': str,
        'msgSource': str,
        'gatewayMacBest': str,
        'occupied': 'Int64',
        'tempC': float,
        'humidity': float,
        'co2': float,
        'sound': float,
        'lux': float
    }
    
    # Read the CSV file with ";" as separator and specified data types
    df = pd.read_csv(file_path, sep=';', dtype=dtype, low_memory=False)
    
    # Print the column names to debug
    print(f"Columns in {file_path}: {df.columns.tolist()}")
    
    # Filter to include only desk sensors
    df = df[df['application'].str.contains('desk', case=False)]
    
    # Convert time to datetime format
    df['time'] = pd.to_datetime(df['time'], format='%Y-%m-%d %H:%M:%S,%f')
    
    # Remove rows before the date extracted from the filename
    df = df[df['time'].dt.date >= file_date]
    
    # Filter out heartbeat messages
    df = df[df['fCnt'].diff() != 0]
    
    # Adjust unoccupancy timestamps
    df.loc[df['occupied'] == 0, 'time'] -= timedelta(minutes=30)
    
    return df

# Function to calculate occupancy for 15-minute intervals
def calculate_occupancy(df):
    # Create a new DataFrame to store occupancy data
    occupancy_data = []
    
    # Get the start and end times
    start_time = df['time'].min().replace(minute=0, second=0, microsecond=0)
    end_time = df['time'].max().replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    
    # Iterate over 15-minute intervals
    current_time = start_time
    while current_time < end_time:
        next_time = current_time + timedelta(minutes=15)
        
        # Filter data for the current interval
        interval_data = df[(df['time'] >= current_time) & (df['time'] < next_time)]
        
        # Calculate occupancy ratio
        occupied_time = interval_data[interval_data['occupied'] == 1]['time'].count()
        total_time = interval_data['time'].count()
        occupancy_ratio = occupied_time / total_time if total_time > 0 else 0
        
        # Append to occupancy data
        occupancy_data.append({
            'interval_start': current_time,
            'interval_end': next_time,
            'occupancy_ratio': occupancy_ratio
        })
        
        # Move to the next interval
        current_time = next_time
    
    return pd.DataFrame(occupancy_data)

# Process CSV files only for the year 2025
year_path = sensor_export_directory

# Check if the year folder exists
if not os.path.exists(year_path):
    print(f"Directory {year_path} does not exist.")
else:
    print(f"Processing files in directory: {year_path}")
    all_data = []
    for file_name in os.listdir(year_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(year_path, file_name)
            print(f"Processing file: {file_path}")
            daily_data = process_csv(file_path)
            all_data.append(daily_data)

    # Concatenate all data
    if all_data:
        all_data_df = pd.concat(all_data)

        # Calculate occupancy for 15-minute intervals
        occupancy_df = calculate_occupancy(all_data_df)

        # Display the occupancy data
        print(occupancy_df.head())
    else:
        print("No CSV files found for the year 2025.")


Processing files in directory: sensor_export_chris/NL-UTC-UTRECHTLROF_2025
Processing file: sensor_export_chris/NL-UTC-UTRECHTLROF_2025\export_assetData_NL-UTC-UTRECHTLROF_20250101.csv
Columns in sensor_export_chris/NL-UTC-UTRECHTLROF_2025\export_assetData_NL-UTC-UTRECHTLROF_20250101.csv: ['sep=', 'Unnamed: 1']


KeyError: 'application'